Injection code
    
    import pickle
    with open('/tmp/data.pkl', 'wb') as f:
        data = dict(
            parameters=parameters,
            filenames=filenames,
        )
        pickle.dump(data, f)

    from os import execvp
    os.environ['BAZEL_RUNFILES'] = os.getcwd()
    os.environ['PATH'] = '/home/eacousineau/bin:/home/eacousineau/.local/bin:'
    sys.path.insert(0, os.path.dirname(__file__))
    os.environ['PYTHONPATH'] = ':'.join(sys.path)
    execvp('jupyter', ['jupyter', 'notebook'])
    exit(1)


In [1]:
import os
os.chdir(os.environ['BAZEL_RUNFILES'])

In [2]:
from mkdoc import *
from clang.cindex import *

In [3]:
import pickle

In [4]:
with open('/tmp/data.pkl', 'rb') as f:
    data = pickle.load(f)

In [5]:
filenames = data['filenames']
parameters = data['parameters']

In [6]:
quiet = False
if True:
    includes = list(map(drake_genfile_path_to_include_path, filenames))
    include_map = FileDict(zip(filenames, includes))
    # TODO(eric.cousineau): Sort files based on include path?
    with NamedTemporaryFile('w') as include_file:
        for include in includes:
            include_file.write("#include \"{}\"\n".format(include))
        include_file.flush()
        if not quiet:
            print("Parse header...", file=sys.stderr)
        index = cindex.Index(
            cindex.conf.lib.clang_createIndex(False, True))
        tu = index.parse(include_file.name, parameters)

    if not quiet:
        print("Extract relevant symbols...", file=sys.stderr)
    output = []
    extract(include_map, tu.cursor, '', output)

Parse header...
Extract relevant symbols...


In [7]:
symbols = sorted(output, key=Symbol.sorting_key)
sym_map = {"{}:{}".format(s.include, s.line): s for s in symbols}

In [8]:
def pretty(n):
    if not n:
        return " x "
    if n.kind == CursorKind.TRANSLATION_UNIT:
        inc = '<>'
    else:
        inc = include_map[d(n.location.file.name)]
    return "{}:{}: {}  ({}, comment: {})".format(inc, n.location.line, d(n.spelling), n.kind, len(n.raw_comment or ''))

In [10]:
ps = [
    [
        ('bad class', sym_map['drake/multibody/ik_options.h:6']),
        ('good class', sym_map['drake/multibody/rigid_body_tree.h:109']),
    ],
    [
        ('bad method', sym_map['drake/multibody/kinematics_cache-inl.h:71']),
        ('good method', sym_map['drake/multibody/kinematics_cache.h:152']),
    ],
    [
        ('good method', sym_map['drake/multibody/rigid_body_tree.h:161']),
    ]
]
for p in ps:
    for label, s in p:
        print(label)
        n = s.node
        print(pretty(n))
        print(n.is_definition(), n.kind.is_reference())
        print(n.get_usr())
        print(pretty(n.canonical))
        print(pretty(n.referenced))
        print(pretty(n.get_definition()))
        print("---")
    print("\n\n")

bad class
drake/multibody/ik_options.h:6: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
False False
b'c:@ST>1#T@RigidBodyTree'
drake/multibody/ik_options.h:6: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
drake/multibody/ik_options.h:6: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
drake/multibody/rigid_body_tree.h:109: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
---
good class
drake/multibody/rigid_body_tree.h:109: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
True False
b'c:@ST>1#T@RigidBodyTree'
drake/multibody/ik_options.h:6: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
drake/multibody/rigid_body_tree.h:109: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
drake/multibody/rigid_body_tree.h:109: RigidBodyTree  (CursorKind.CLASS_TEMPLATE, comment: 1577)
---



bad method
drake/multibody/kinematics_cache-inl.h:71: get_mutable_element  (CursorKind.CXX_METHOD, comment: 65)
True False
b'c:@ST>1#T@Kinemati

In [11]:
s = sym_map['drake/common/trajectories/piecewise_quaternion.h:82']
pretty(s.node)

'drake/common/trajectories/piecewise_quaternion.h:82: orientation  (CursorKind.CXX_METHOD, comment: 127)'

In [ ]:
print_symbols(output)

In [12]:
get_full_name(s.node)

'drake::trajectories::PiecewiseQuaternionSlerp::orientation'

In [ ]:
for s in output:
    print("'{}'".format(get_full_name(s.node)))

In [ ]:
s = sym_map["drake/geometry/geometry_instance.h:79"]
n = s.node

In [ ]:
print(pretty(n))
print(pretty(n.lexical_parent))

In [ ]:
n.extent